In [2]:
from calendar import month
from operator import index
import snscrape.modules.twitter as sntwitter
import pandas as pd
import glob
import datetime

release_dates = pd.read_csv("Release dates - data.csv")
release_dates_map = {}

folderpathrt = "rotten_info/*"
filesrt = []

for file in glob.glob(folderpathrt):
    filesrt.append(file)

for file in filesrt:
    print(file)
    movies = pd.read_csv(file)

    tweets = []
    for ind, movie in movies.iterrows():
        movien = movie['Release Group']
        if not pd.isna(movie['Release Date (Theaters)']):
            release_date = ('').join(movie['Release Date (Theaters)'].strip('[]').split(',')).replace("'", "")
            rd = datetime.datetime.strptime(release_date, "%b %d %Y")
            # if release_dates_map.get(movien.lower(), ''):
            release_dates_map[movien.lower()] = rd

folderpath = "Box Office Mojo/*"
files = []

for file in glob.glob(folderpath):
    files.append(file)

for ind, row in release_dates.iterrows():
    if row['Release-Date']:
        rd = datetime.datetime.strptime(row['Release-Date'], "%Y-%m-%d")
        if not release_dates_map.get(row['Movie'].lower(), None):
            release_dates_map[row['Movie'].lower()] = rd
    
count = 0
c = 0
for file in files:
    if '2014' in file:
        print(file)
        movies = pd.read_csv(file)

        tweets = []
        for ind, movie in movies.iterrows():
            c += 1
            query_txt = movie['Release Group']
            release_date = release_dates_map.get(movie['Release Group'].lower(), None)
            if release_date:
                count += 1
                # rd = datetime.datetime.strptime(release_date, "%Y-%m-%d")
                since = release_date - datetime.timedelta(weeks=4)
                until = release_date + datetime.timedelta(weeks=1)
                query_txt += " since:%s until:%s" % (since.date(), until.date())
                print(movie['Release Group'], release_date.date())

                for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query_txt).get_items()):
                    if i>5000:
                        break
                    tweets.append([movie['Release Group'], tweet.date, tweet.id, tweet.content, tweet.user.username])

        tweets_df = pd.DataFrame(tweets, columns=['Movie Name', 'Datetime', 'Tweet Id', 'Text', 'Username'])
        tweets_df.to_csv("Twitter/tweets_%s" % file.split('/')[1], index=False, escapechar=r'|')
        break


Box Office Mojo/2014.csv
Transformers: Age of Extinction 2014-06-27
The Hobbit: The Battle of the Five Armies 2014-12-17
Guardians of the Galaxy 2014-08-01
Maleficent 2014-05-30
The Hunger Games: Mockingjay - Part 1 2014-11-21
X-Men: Days of Future Past 2014-05-23
Captain America: The Winter Soldier 2014-04-04
Dawn of the Planet of the Apes 2014-07-11
The Amazing Spider-Man 2 2014-05-02
Interstellar 2014-11-05
Big Hero 6 2014-11-07
How to Train Your Dragon 2 2014-06-13
American Sniper 2015-12-25
Godzilla 2014-05-16
Rio 2 2014-04-11
Teenage Mutant Ninja Turtles 2014-08-08
The Lego Movie 2014-02-07
Lucy 2014-07-25
Penguins of Madagascar 2014-11-26
Edge of Tomorrow 2014-06-06
Gone Girl 2014-10-03
Night at the Museum: Secret of the Tomb 2014-12-19
Noah 2014-03-28
The Maze Runner 2014-09-19
300: Rise of an Empire 2014-03-07
22 Jump Street 2014-06-13
The Fault in Our Stars 2014-06-06
Divergent 2014-03-21
Paddington 2015-01-16
Mr. Peabody & Sherman 2014-03-07
Neighbors 2014-05-09
Exodus: Gods